Разработка простых веб-приложений на различных фреймворках — отличный способ понять особенности каждого из них. Мы рассмотрим три популярных фреймворка для создания веб-приложений на Python: Django, Flask и FastAPI. Для сравнения, разработаем одно и то же приложение на каждом из них, а затем проведем анализ их особенностей.

**Задача**

Разработаем простое веб-приложение, которое выполняет следующие функции:

1. Отображает главную страницу с формой для ввода текста.
2. Принимает текст от пользователя и сохраняет его в базе данных.
3. Отображает список всех сохраненных текстов.

Для начала давайте определим структуру базы данных. Она будет содержать одну таблицу с двумя полями: идентификатор записи и сам текст.

***DJANGO***

Django — это полнофункциональный фреймворк, ориентированный на быструю разработку сложных веб-приложений. Он предоставляет встроенную ORM, систему аутентификации, админ-панель и многое другое.

Документация: https://docs.djangoproject.com/en/5.1/

Установка



In [1]:
pip install django

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 8.3 MB/s eta 0:00:00


Создание проекта

In [2]:
django-admin startproject mysite
cd mysite

SyntaxError: invalid syntax (<ipython-input-2-7e3c9ebd605f>, line 1)

Создание приложения

In [ ]:
python manage.py startapp main_app

Добавляем main_app в список установленных приложений в mysite/settings.py:

In [ ]:
INSTALLED_APPS = [
    # ...
    'main_app',
]

  **Модель**
  
Создадим модель для хранения текста в main_app/models.py:

In [ ]:
from django.db import models

class Text(models.Model):
    text = models.TextField()

    def __str__(self):
        return self.text[:50]

Выполним миграцию базы данных:

In [ ]:
python manage.py makemigrations
python manage.py migrate

**Представления**

Определим представления в main_app/views.py:

In [ ]:
from django.shortcuts import render, redirect
from .models import Text
from .forms import TextForm

def index(request):
    texts = Text.objects.all().order_by('-id')
    form = TextForm()
    context = {'texts': texts, 'form': form}
    return render(request, 'main_app/index.html', context)

def save_text(request):
    if request.method == 'POST':
        form = TextForm(request.POST)
        if form.is_valid():
            form.save()
            return redirect('index')
    return redirect('index')

**Формы**

Создадим форму для ввода текста в main_app/forms.py:

In [ ]:
from django import forms
from .models import Text

class TextForm(forms.ModelForm):
    class Meta:
        model = Text
        fields = ['text']

**Шаблоны**

Создадим шаблон главной страницы в main_app/templates/main_app/index.html:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Django App</title>
</head>
<body>
    <h1>Главная страница</h1>
    <form action="{% url 'save_text' %}" method="post">
        {% csrf_token %}
        {{ form }}
        <button type="submit">Сохранить</button>
    </form>
    <hr>
    <h2>Список текстов</h2>
    <ul>
        {% for text in texts %}
            <li>{{ text }}</li>
        {% endfor %}
    </ul>
</body>
</html>

**URLs**

Определим маршруты в mysite/urls.py:

In [ ]:
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('main_app.urls')),
]

И создаем файл urls.py в директории main_app:

In [ ]:
from django.urls import path
from .views import index, save_text

urlpatterns = [
    path('', index, name='index'),
    path('save-text/', save_text, name='save_text'),
]

Теперь запустим сервер:

In [ ]:
python manage.py runserver

Откроется главная страница по адресу http://127.0.0.1:8000/, где можно вводить тексты и сохранять их.